In [ ]:
import numpy as np
import pandas as pd

import sqlite3 as sql

In [ ]:
db = sql.connect('northwind.db')

In [ ]:
# query di bawah akan menampilkan tabel - tabel yang ada di database kita
query = """
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""

In [ ]:
# buat variabel cursor dari koneksi ke database kita
cursor = db.cursor()

In [ ]:
# menjalankan query dan menampilkan hasil
cursor.execute(query)
print(cursor.fetchall())

[('Suppliers',), ('Shippers',), ('Employees',), ('Customers',), ('Orders',), ('Categories',), ('Products',), ('OrderDetails',)]


In [ ]:
pd.read_sql("SELECT * FROM Employees LIMIT 2;", db)

,EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath
0,1,Davolio,Nancy,Sales Representative,Ms.,1966-12-08 00:00:00.000,2010-05-01 00:00:00.000,507 - 20th Ave. E.\nApt. 2A,Seattle,WA,98122,USA,(206) 555-9857,5467,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Education includes a BA in psychology from Col...,2.0,http://accweb/emmployees/davolio.bmp
1,2,Fuller,Andrew,"Vice President, Sales",Dr.,1970-02-19 00:00:00.000,2010-08-14 00:00:00.000,908 W. Capital Way,Tacoma,WA,98401,USA,(206) 555-9482,3457,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Andrew received his BTS commercial in 1974 and...,NaN,http://accweb/emmployees/fuller.bmp


## 1. WINDOW FUNCTION

### 1.1 Window Function berdasarkan kelompok tertentu

Misalkan kita mau membandingkan apakah harga suatu produk lebih tinggi atau lebih rendah dari rata-rata harga produk di kategori produk tersebut

In [ ]:
# kita bisa gunakan window function dengan tambahan klausa PARTITION BY
query = """
SELECT
  ProductName,
  CategoryID,
  UnitPrice,
  AVG(UnitPrice) OVER(PARTITION BY CategoryID) AS avg_price_per_category
FROM products

"""
pd.read_sql(query, db)

,ProductName,CategoryID,UnitPrice,avg_price_per_category
0,Chai,1,18.00,37.979167
1,Chang,1,19.00,37.979167
2,Guaraná Fantástica,1,4.50,37.979167
3,Sasquatch Ale,1,14.00,37.979167
4,Steeleye Stout,1,18.00,37.979167
...,...,...,...,...
72,Jack's New England Clam Chowder,8,9.65,20.682500
73,Rogede sild,8,9.50,20.682500
74,Spegesild,8,12.00,20.682500
75,Escargots de Bourgogne,8,13.25,20.682500


In [ ]:
# sebagai validasi mari kita coba cari harga rata-rata per categoryID
query = """
SELECT
  CategoryID,
  AVG(UnitPrice)
FROM products
GROUP BY 1

"""
pd.read_sql(query, db)

,CategoryID,AVG(UnitPrice)
0,1,37.979167
1,2,23.062500
2,3,25.160000
3,4,28.730000
4,5,20.250000
5,6,54.006667
6,7,32.370000
7,8,20.682500


Terlihat bahwa hasil window function kita konsisten dengan saat kita melakukan grouping berdasarkan categoryID

### 1.2 RANK VS DENSE_RANK VS ROW_NUMBER

Mari kita buat ranking untuk tiap produk berdasarkan harganya

In [ ]:
query = """
SELECT
  ProductName,
  UnitPrice,
  RANK() OVER(ORDER BY UnitPrice) AS RANK_,
  DENSE_RANK() OVER(ORDER BY UnitPrice) AS DENSE_RANK_,
  ROW_NUMBER() OVER(ORDER BY UnitPrice) AS ROW_NUMBER_
FROM products

"""
pd.read_sql(query, db)

,ProductName,UnitPrice,RANK_,DENSE_RANK_,ROW_NUMBER_
0,Geitost,2.50,1,1,1
1,Guaraná Fantástica,4.50,2,2,2
2,Konbu,6.00,3,3,3
3,Filo Mix,7.00,4,4,4
4,Tourtière,7.45,5,5,5
...,...,...,...,...,...
72,Carnarvon Tigers,62.50,73,58,73
73,Sir Rodney's Marmalade,81.00,74,59,74
74,Mishi Kobe Niku,97.00,75,60,75
75,Thüringer Rostbratwurst,123.79,76,61,76


Kita bisa juga buat perankingan berdasarkan categoryID nya

In [ ]:
query = """
SELECT
  ProductName,
  UnitPrice,
  CategoryID,
  RANK() OVER(PARTITION BY CategoryID ORDER BY UnitPrice) AS RANK_,
  DENSE_RANK() OVER(PARTITION BY CategoryID ORDER BY UnitPrice) AS DENSE_RANK_,
  ROW_NUMBER() OVER(PARTITION BY CategoryID ORDER BY UnitPrice) AS ROW_NUMBER_
FROM products

"""
pd.read_sql(query, db)

,ProductName,UnitPrice,CategoryID,RANK_,DENSE_RANK_,ROW_NUMBER_
0,Guaraná Fantástica,4.50,1,1,1,1
1,Rhönbräu Klosterbier,7.75,1,2,2,2
2,Sasquatch Ale,14.00,1,3,3,3
3,Laughing Lumberjack Lager,14.00,1,3,3,4
4,Outback Lager,15.00,1,5,4,5
...,...,...,...,...,...,...
72,Inlagd Sill,19.00,8,8,8,8
73,Nord-Ost Matjeshering,25.89,8,9,9,9
74,Gravad lax,26.00,8,10,10,10
75,Ikura,31.00,8,11,11,11


### CASE WHEN

Mari kita buat sebuah kolom untuk mengkategorikan suatu order berdasarkan quantity dari tabel orderdetails

In [ ]:
# kita lakukan pengelompokan berdasarkan quantitynya
query = """
SELECT OrderID, ProductID,Quantity,
CASE
    WHEN Quantity >= 30 THEN 'jumlah>=30'
    ELSE 'jumlah < 30'
END AS QuantityText
FROM OrderDetails;
"""
pd.read_sql(query, db)

,OrderID,ProductID,Quantity,QuantityText
0,10248,11,12,jumlah < 30
1,10248,42,10,jumlah < 30
2,10248,72,5,jumlah < 30
3,10249,14,9,jumlah < 30
4,10249,51,40,jumlah>=30
...,...,...,...,...
2150,11077,64,2,jumlah < 30
2151,11077,66,1,jumlah < 30
2152,11077,73,2,jumlah < 30
2153,11077,75,4,jumlah < 30


In [ ]:
# mari kita coba mengkategorikan umur para pegawai di northwind
# dengan 3 kategori : usia 25 - 35, usia 36 - 45, dan usia > 45

query = """

SELECT LastName
    , FirstName
    , hiredate - birthdate AS Age
    , CASE WHEN (hiredate - birthdate) >= 25 and (hiredate - birthdate) < 36 then 'usia 25-35'
           WHEN (hiredate - birthdate) >= 36 and (hiredate - birthdate) < 46 then 'usia 36-45'
           ELSE 'usia > 45' END as kelompok_usia
FROM Employees
"""
pd.read_sql_query(query, db)

,LastName,FirstName,Age,kelompok_usia
0,Davolio,Nancy,44,usia 36-45
1,Fuller,Andrew,40,usia 36-45
2,Leverling,Janet,29,usia 25-35
3,Peacock,Margaret,56,usia > 45
4,Buchanan,Steven,38,usia 36-45
5,Suyama,Michael,30,usia 25-35
6,King,Robert,34,usia 25-35
7,Callahan,Laura,36,usia 36-45
8,Dodsworth,Anne,28,usia 25-35


# Mini Case

Selamat ! Kamu baru saja diterima bekerja sebagai seorang Jr. Data Analyst di perusahaan Nortwind Traders!. Hari ini adalah hari pertama kamu bekerja, dan manager kamu ingin kamu secara perlahan mengetahui seluk beluk dari perusahaan tempat kamu bekerja. Kamu diminta untuk mengerjakan beberapa tugas berikut menggunakan bantuan SQL yang sudah kamu pelajari sebelumnya.
Tugas kamu adalah membuat query untuk menjawab beberapa tugas yang akan diberikan di bawah ini, kemudian kamu dapat memberikan komentar sesuai dengan hasil query kamu.
Selamat bekerja!!

1. Dari tabel customers, kelompokkan lah customer berdasarkan benua mereka. Ketentuannya adalah jika negara asal adalah USA atau Canada, maka benuanya adalah North America. Jika berasal dari Argentina, Brazil, Venezuela maka benuanya adalah South America. Selain itu maka benua Eropa. Kemudian hitung jumlah customer dari tiap benua.

2. Dari tabel orderdetails kita bisa mengetahui total sales untuk setiap orderan yang masuk ke northwind dengan cara mengalikan jumlah barang terjual dan harga barangnya. Tampilkan total sales untuk order 10248, 10249, dan 11077, kemudian buat ranking dari total sales untuk setiap order id